# Plot FVCOM netcdf files
**Author: Jun Sasaki Coded on 2024-12-26 Updated on 2024-12-29**<br>
Checking input netcdf files by creating a new xfvcom.py, which should be published as a package.

In [ ]:
import xarray as xr
import os
from xfvcom import FvcomDataLoader, FvcomAnalyzer, FvcomPlotConfig, FvcomPlotter
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from math import ceil

## Plot output netcdf

In [ ]:
base_path = "~/Github/TB-FVCOM/goto2023/output"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay18_r16_crossed_0001.nc"]
index_ncfile=0
ncfile_path = f"{base_path}/{ncfiles[index_ncfile]}"
ofc = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
ofc.ds

In [ ]:
start=None; end=None
index=1  # node or nele number
k=0 # siglay or siglev number
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ofc.ds, plot_config)
var_name = 'viscofm'
save_path = f"plot_{var_name}.png"
plotter.plot_timeseries(var_name, log=True, index=index, k=k,start=start, end=end, save_path=save_path)

In [ ]:
plotter = FvcomPlotter(ofc.ds, plot_config)
var_name = 'temp'
save_path = f"plot_{var_name}.png"
ax=plotter.plot_timeseries_2d(var_name, index=index, start=start, end=end, ylim=(-4,0), depth=True,
                              #levels=[9.8, 9.85, 9.9, 9.95, 10.0, 10.05, 10.1, 10.15],
                              rolling_window=0, cmap="coolwarm", vmin=None, vmax=10.2,
                              method='contourf', add_contour=False, label_contours=False,
                              save_path=save_path)

## Plot input netcdf files

In [ ]:
base_path = "~/Github/TB-FVCOM/goto_dye/input/input_steady/2020"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay18_2020_wnd.nc","TokyoBay2020final_tsobc.nc", "TokyoBay2020julian_obc.nc",
           "TokyoBay2020kisarazufinal_sewer.nc", "TokyoBay2020final_river.nc",
           "TokyoBay2020final_sewer.nc"]
index_ncfiles=0
ncfile_path = f"{base_path}/{ncfiles[index_ncfile]}"
fvcom = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
fvcom.ds

In [ ]:
start='2020-01-01'; end='2020-01-20'

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(fvcom.ds, plot_config)
var_name = 'elevation'
save_path = f"plot_{var_name}.png"
plotter.plot_timeseries(var_name, 1, start=start, end=end, save_path=save_path)

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(fvcom.ds, plot_config)
# ノード0の風速ベクトル時系列を2020年1月1日から12月31日までプロット
plotter.plot_wind_vector_timeseries(nele=0, start=start, end=end, rolling_window=25,
                                    save_path="wind_vector_plot.png"
                                   )

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(fvcom.ds, plot_config)
ax=plotter.plot_timeseries('long_wave', index=0, start=start, end=end, rolling_window=25)

In [ ]:
analyzer=FvcomAnalyzer(fvcom.ds)
vars = analyzer.get_variables_by_dims(("time", "node"))
print(vars)

In [ ]:
plot_config = FvcomPlotConfig(figsize=(8, 2))
plotter = FvcomPlotter(fvcom.ds, plot_config)

# HelperMixin のメソッドを利用
plotter.plot_timeseries_in_batches(
    plotter=plotter,
    vars=vars,
    index=0,
    batch_size=5,
    start=start,
    end=end,
    rolling_window=25,
    save_prefix="time_node"
)

In [ ]:
vars = analyzer.get_variables_by_dims(("time", "nele"))
print(vars)

In [ ]:
plot_config = FvcomPlotConfig(figsize=(8, 2))
plotter = FvcomPlotter(fvcom.ds, plot_config)

# HelperMixin のメソッドを利用
plotter.plot_timeseries_in_batches(
    plotter=plotter,
    vars=vars,
    index=0,
    batch_size=5,
    start=start,
    end=end,
    rolling_window=25,
    save_prefix="time_nele"
)

In [ ]:
plot_config = FvcomPlotConfig(figsize=(8, 2))
plotter = FvcomPlotter(fvcom.ds, plot_config)

plotter.plot_timeseries_for_river(
    var_name="river_flux",
    river_index=0,
    start=start,
    end=end,
    rolling_window=25,
    save_path="river_discharge.png"
)

In [ ]:
plot_config = FvcomPlotConfig(figsize=(8, 2))
plotter = FvcomPlotter(fvcom.ds, plot_config)
var_name='river_temp'
# HelperMixin のメソッドを利用
plotter.plot_timeseries_for_river_in_batches(
    plotter=plotter,
    var_name=var_name,
    batch_size=5,
    start=start,
    end=end,
    rolling_window=25,
    save_prefix=var_name
)

## Example for input netcdf files

In [ ]:
base_path = "~/Github/TB-FVCOM/goto2023/input"
base_path = os.path.expanduser(base_path)
ncfiles = ["TokyoBay_tsobc.nc", "TokyoBay_julian_obc.nc","TokyoBayfinal_river.nc", "TokyoBayfinal16_river.nc",
           "TokyoBay18_z0.nc", "TokyoBay18_wnd.nc",
           "TokyoBay2020final_sewer.nc"]
index_ncfiles=5
ncfile_path = f"{base_path}/{ncfiles[index_ncfile]}"
ifc = FvcomDataLoader(ncfile_path=ncfile_path, time_tolerance=5)
ifc.ds

## Compare with meteorological data
- Needs to install metdata; see [GitHub](https://github.com/jsasaki-utokyo/metdata)

In [ ]:
from metdata import gwo

datetime_ini = "2020-01-01 00:00:00"
datetime_end = "2021-01-01 00:00:00"
stn = "Chiba"
base_path = "/mnt/c/dat/met/JMA_DataBase/GWO/Hourly/"

met = gwo.Hourly(datetime_ini=datetime_ini, datetime_end=datetime_end, stn=stn, dirpath=base_path)
met.df.head()  # pandas.DataFrame

In [ ]:
met_ds = xr.Dataset.from_dataframe(met.df)
met_ds = met_ds.rename_dims({"index": "time"}).rename_vars({"index": "time"})
met_ds

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(met_ds, plot_config)
start='2020-01-01'; end='2020-01-31'
plotter.plot_wind_vector_timeseries(u_var='u', v_var='v', nele=None, start=start, end=end, rolling_window=1,
                                    save_path="wind_vector_plot.png")

In [ ]:
plot_config = FvcomPlotConfig(width=8, height=2)
plotter = FvcomPlotter(ifc.ds, plot_config)
# ノード0の風速ベクトル時系列を2020年1月1日から12月31日までプロット
plotter.plot_wind_vector_timeseries(nele=0, start=start, end=end, rolling_window=1,
                                    save_path="wind_vector_plot.png")